In [2]:
%pip install pandas sqlalchemy langchain langchain_community langchain_openai python_dotenv  --default-timeout=100

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

df = pd.read_csv("./F1.csv")

AttributeError: partially initialized module 'pandas' from 'c:\Users\salva\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\__init__.py' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

# Creación de base de datos SQL a partir de dataframe (csv)

In [ ]:
import sqlite3
from sqlalchemy import create_engine

engine = create_engine('sqlite:///f1.db', echo=True)
df.to_sql('f1', con=engine, if_exists='replace', index=False)

2025-07-26 11:36:23,778 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-26 11:36:23,893 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("f1")
2025-07-26 11:36:23,898 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-26 11:36:23,904 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("f1")
2025-07-26 11:36:23,906 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-26 11:36:23,936 INFO sqlalchemy.engine.Engine 
CREATE TABLE f1 (
	"raceId" BIGINT, 
	year BIGINT, 
	round BIGINT, 
	"circuitId" BIGINT, 
	name_x TEXT, 
	date TEXT, 
	time_x TEXT, 
	"circuitRef" TEXT, 
	name_y TEXT, 
	location TEXT, 
	country TEXT, 
	lat FLOAT, 
	lng FLOAT, 
	alt BIGINT, 
	"resultId" BIGINT, 
	"driverId" BIGINT, 
	"constructorId" BIGINT, 
	number_x FLOAT, 
	grid BIGINT, 
	position FLOAT, 
	"positionText" TEXT, 
	"positionOrder" BIGINT, 
	points FLOAT, 
	laps BIGINT, 
	time_y TEXT, 
	milliseconds FLOAT, 
	"fastestLap" FLOAT, 
	rank FLOAT, 
	"fastestLapTime" TEXT, 
	"fastestLapSpeed" FLOAT, 
	"status

25121

# Creación de agente
- La temperatura es 0 para que se limite a responder con la información de la base de datos y que no se ponga creativo

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

#from langchain_openai import ChatOpenAI #OpenAI LLM
from langchain_google_genai import ChatGoogleGenerativeAI #Google LLM
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType

db = SQLDatabase(engine=engine)

# llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY")) #openai llm
#agente = create_sql_agent(llm=llm, database=db,agent_type="openai-tools" , verbose=True) #verbose=True para ver como "piensa" el agente

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.8, google_api_key=os.getenv("GOOGLE_API_KEY")) #google llm
agente = create_sql_agent(llm=llm, db=db, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True) 

NameError: name 'engine' is not defined

# Algunos ejemplos de consultas

In [ ]:
response = agente.invoke({"input":"¿cuantos pilotos diferentes hay en la base de datos?"})
print(response['output'])



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:f1I should look at the schema of the f1 table.
Action: sql_db_schema
Action Input: f12025-07-26 11:47:32,304 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-26 11:47:32,306 INFO sqlalchemy.engine.Engine SELECT f1."raceId", f1.year, f1.round, f1."circuitId", f1.name_x, f1.date, f1.time_x, f1."circuitRef", f1.name_y, f1.location, f1.country, f1.lat, f1.lng, f1.alt, f1."resultId", f1."driverId", f1."constructorId", f1.number_x, f1.grid, f1.position, f1."positionText", f1."positionOrder", f1.points, f1.laps, f1.time_y, f1.milliseconds, f1."fastestLap", f1.rank, f1."fastestLapTime", f1."fastestLapSpeed", f1."statusId", f1."constructorRef", f1.name, f1.nationality_x, f1."driverRef", f1.number_y, f1.code, f1.forename, f1.surname, f1.dob, f1.nationality_y, f1.race_datetime, f1.result_datetime, f1.race_year, f1.race_month, f1.race_day, f1.race_weekday, f1.race_hour, f1.birth_date, f1.driver_age, f1."fast

In [ ]:
response = agente.invoke({"input":"¿Cuántos victorias tiene Ayrton Senna?"})
print(response['output'])



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:f1The only table is f1. I should look at the schema of the f1 table.
Action: sql_db_schema
Action Input: f12025-07-26 11:47:11,414 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-26 11:47:11,415 INFO sqlalchemy.engine.Engine SELECT f1."raceId", f1.year, f1.round, f1."circuitId", f1.name_x, f1.date, f1.time_x, f1."circuitRef", f1.name_y, f1.location, f1.country, f1.lat, f1.lng, f1.alt, f1."resultId", f1."driverId", f1."constructorId", f1.number_x, f1.grid, f1.position, f1."positionText", f1."positionOrder", f1.points, f1.laps, f1.time_y, f1.milliseconds, f1."fastestLap", f1.rank, f1."fastestLapTime", f1."fastestLapSpeed", f1."statusId", f1."constructorRef", f1.name, f1.nationality_x, f1."driverRef", f1.number_y, f1.code, f1.forename, f1.surname, f1.dob, f1.nationality_y, f1.race_datetime, f1.result_datetime, f1.race_year, f1.race_month, f1.race_day, f1.race_weekday, f1.race_hour, f1.birth_date, f

In [ ]:
response = agente.invoke({"input":"¿Qué porcentaje de carreras se corrieron bajo la lluvia?"})
print(response['output'])



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:f1I should look at the schema of the f1 table.
Action: sql_db_schema
Action Input: f12025-07-26 11:48:06,174 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-26 11:48:06,175 INFO sqlalchemy.engine.Engine SELECT f1."raceId", f1.year, f1.round, f1."circuitId", f1.name_x, f1.date, f1.time_x, f1."circuitRef", f1.name_y, f1.location, f1.country, f1.lat, f1.lng, f1.alt, f1."resultId", f1."driverId", f1."constructorId", f1.number_x, f1.grid, f1.position, f1."positionText", f1."positionOrder", f1.points, f1.laps, f1.time_y, f1.milliseconds, f1."fastestLap", f1.rank, f1."fastestLapTime", f1."fastestLapSpeed", f1."statusId", f1."constructorRef", f1.name, f1.nationality_x, f1."driverRef", f1.number_y, f1.code, f1.forename, f1.surname, f1.dob, f1.nationality_y, f1.race_datetime, f1.result_datetime, f1.race_year, f1.race_month, f1.race_day, f1.race_weekday, f1.race_hour, f1.birth_date, f1.driver_age, f1."fast

In [ ]:
response = agente.invoke({"input":"¿cuantos pares son 3 botas?"})
print(response['output'])



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:f1The only table is f1. I should look at the schema of f1.
Action: sql_db_schema
Action Input: f12025-07-26 11:54:27,938 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-26 11:54:27,940 INFO sqlalchemy.engine.Engine SELECT f1."raceId", f1.year, f1.round, f1."circuitId", f1.name_x, f1.date, f1.time_x, f1."circuitRef", f1.name_y, f1.location, f1.country, f1.lat, f1.lng, f1.alt, f1."resultId", f1."driverId", f1."constructorId", f1.number_x, f1.grid, f1.position, f1."positionText", f1."positionOrder", f1.points, f1.laps, f1.time_y, f1.milliseconds, f1."fastestLap", f1.rank, f1."fastestLapTime", f1."fastestLapSpeed", f1."statusId", f1."constructorRef", f1.name, f1.nationality_x, f1."driverRef", f1.number_y, f1.code, f1.forename, f1.surname, f1.dob, f1.nationality_y, f1.race_datetime, f1.result_datetime, f1.race_year, f1.race_month, f1.race_day, f1.race_weekday, f1.race_hour, f1.birth_date, f1.driver_a